In [232]:
import pandas as pd
import numpy as np
import unicodedata
from unidecode import unidecode

# **Preprocessing user_info dataset**

In [233]:
user_info = pd.read_csv(r'data/user_info.csv')
user_info.head()

,user_id,age,sex,phone,job,carrier,marital_status
0,376517,44.0,gentle,******6654,Thực tập sinh giáo dục,other,Married
1,234512,39.0,lady,NaN,NaN,vietnamobile,Married
2,344532,39.0,she,******6296,DevOps Engineer,other,cưới
3,186135,37.0,she,******1502,Thực tập sinh giáo dục,vietnamobile,cưới
4,30230,38.0,female,******4966,DevOps Engineer,viettel,Married


In [234]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         424170 non-null  int64  
 1   age             328803 non-null  float64
 2   sex             418652 non-null  object 
 3   phone           402962 non-null  object 
 4   job             402962 non-null  object 
 5   carrier         402962 non-null  object 
 6   marital_status  423813 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 22.7+ MB


In [235]:
# Check ratio of missing values all columns
missing_values = round(user_info.isnull().sum() / len(user_info), 2)
missing_values

user_id           0.00
age               0.22
sex               0.01
phone             0.05
job               0.05
carrier           0.05
marital_status    0.00
dtype: float64

## **Preprocessing `age` column**

In [236]:
# Validate that age is an integer greater than 0
user_info[(user_info['age'] < 0) | (user_info['age'] % 1 != 0)]

,user_id,age,sex,phone,job,carrier,marital_status
28,171799,NaN,men,******0902,NaN,vietnamobile,ly hôn
103,32350,NaN,male,******1341,Y tá,vietnamobile,Divorced
132,210917,NaN,male,*****4773,Nhân viên lễ tân,mobiphone,ly hôn
144,38665,NaN,men,*****7182,Luật sư,vietnamobile,Married
146,115159,NaN,gentle,******2492,Chuyên viên bảo mật,mobiphone,Married
...,...,...,...,...,...,...,...
424160,97195,NaN,NaN,*****8530,Nhân viên ngân hàng,mobiphone,ly hôn
424162,156155,NaN,men,*******5301,NaN,viettel,ly hôn
424164,297789,NaN,men,*******0442,Biên dịch viên,viettel,Divorced
424166,245950,NaN,male,*****3449,Tư vấn khách hàng,other,Divorced


In [237]:
# Replace age is not an integer greater than 0 with NaN
user_info.loc[(user_info['age'] < 0) | (user_info['age'] % 1 != 0), 'age'] = np.nan

## **Preprocessing `sex` column**

In [238]:
def normalize_unicode(text):
    if pd.isna(text):
        return text
    text = unicodedata.normalize('NFKC', str(text))
    text = unidecode(text)
    return text.lower().strip()

def sex_mapping(value):
    if pd.isna(value):
        return np.nan
    female_keywords = {'gentle', 'lady', 'she', 'female', 'girl', 'woman', 'f'}
    male_keywords = {'male', 'm', 'boy', 'he', 'men'}
    unknown_keywords = {'unknown', 'other', 'snknown', 'nsll'}
    if value in female_keywords:
        return 'female'
    elif value in male_keywords:
        return 'male'
    elif value in unknown_keywords:
        return np.nan

In [239]:
user_info['sex'].unique()

array(['gentle', 'lady', 'she', 'female', 'girl', 'male', 'M', nan, 'boy',
       'woman', 'F', 'he', 'men', 'unknown', 'other', 'mаle', 'mеn', 'hе',
       'malе', 'lаdy', 'wοman', 'fеmale', 'femаle', 'bοy', 'shе', 'gіrl',
       'womаn', 'gеntle', 'սnknown', 'femalе', 'gentlе', 'othеr', 'οther',
       'unknοwn', 'nսll', 'femаlе', 'fеmalе', 'wοmаn', 'mаlе', 'fеmаle',
       'gеntlе'], dtype=object)

In [240]:
user_info['sex'] = user_info['sex'].apply(normalize_unicode)
user_info['sex'].unique()

array(['gentle', 'lady', 'she', 'female', 'girl', 'male', 'm', nan, 'boy',
       'woman', 'f', 'he', 'men', 'unknown', 'other', 'snknown', 'nsll'],
      dtype=object)

In [241]:
user_info['sex'] = user_info['sex'].apply(sex_mapping)
user_info['sex'].unique()

array(['female', 'male', nan], dtype=object)

## **Preprocessing `phone` column**

In [242]:
# Replace missing values in 'phone' with the nan
user_info['phone'] = user_info['phone'].fillna(np.nan)

## **Preprocessing `job` column**

In [243]:
user_info['job'].unique()

array(['Thực tập sinh giáo dục', nan, 'DevOps Engineer',
       'Kiến trúc sư phần mềm', 'UX/UI Designer',
       'Trưởng phòng kinh doanh', 'Lập trình viên',
       'Nhà phân tích dữ liệu', 'Y tá', 'Quản lý sản xuất',
       'Kỹ sư giám sát', 'Digital Marketer', 'Nghệ sĩ', 'Thực Tập',
       'Điều Dưỡng', 'Kiến trúc sư', 'Nông dân', 'Chuyên viên nhân sự',
       'Nhân viên kinh doanh', 'Nhân viên hành chính', 'Nhà báo',
       'Nhân viên nhà hàng', 'Bác sĩ', 'Chuyên viên tư vấn tuyển sinh',
       'Kỹ sư nông nghiệp', 'Thợ xây', 'Giảng viên',
       'Chuyên viên bảo mật', 'Kỹ sư xây dựng', 'Kỹ sư phần mềm',
       'Nhà thiết kế đồ họa', 'Project Manager', 'Nhà tư vấn tài chính',
       'Nhân viên khách sạn', 'Kiểm toán', 'Thư ký pháp lý',
       'Công nhân sản xuất', 'Nhân viên lễ tân', 'Chuyên viên tín dụng',
       'Giáo viên', 'Sales Representative', 'Tester', 'Kỹ sư sản xuất',
       'Biên dịch viên', 'Luật sư', 'Chuyên viên bán hàng trực tuyến',
       'Chuyên viên phân tích thị 

In [244]:
user_info['job'] = user_info['job'].apply(normalize_unicode)
user_info['job'].unique()

array(['thuc tap sinh giao duc', nan, 'devops engineer',
       'kien truc su phan mem', 'ux/ui designer',
       'truong phong kinh doanh', 'lap trinh vien',
       'nha phan tich du lieu', 'y ta', 'quan ly san xuat',
       'ky su giam sat', 'digital marketer', 'nghe si', 'thuc tap',
       'dieu duong', 'kien truc su', 'nong dan', 'chuyen vien nhan su',
       'nhan vien kinh doanh', 'nhan vien hanh chinh', 'nha bao',
       'nhan vien nha hang', 'bac si', 'chuyen vien tu van tuyen sinh',
       'ky su nong nghiep', 'tho xay', 'giang vien',
       'chuyen vien bao mat', 'ky su xay dung', 'ky su phan mem',
       'nha thiet ke do hoa', 'project manager', 'nha tu van tai chinh',
       'nhan vien khach san', 'kiem toan', 'thu ky phap ly',
       'cong nhan san xuat', 'nhan vien le tan', 'chuyen vien tin dung',
       'giao vien', 'sales representative', 'tester', 'ky su san xuat',
       'bien dich vien', 'luat su', 'chuyen vien ban hang truc tuyen',
       'chuyen vien phan tich thi 

## **Preprocessing `carrier` column**

In [245]:
user_info['carrier'].unique()

array(['other', 'vietnamobile', 'viettel', 'vinaphone', 'mobiphone', nan],
      dtype=object)

In [246]:
user_info['carrier'] = user_info['carrier'].fillna(np.nan)

## **Preprocessing `marital_status` column**

In [247]:
def marital_status_mapping(value):
    if pd.isna(value):
        return np.nan
    married_keywords = ['Married', 'cưới']
    single_keywords = ['Unmarried', 'Single', 'độc thân']
    divorced_keywords = ['Divorced', 'ly hôn']
    if value in married_keywords:
        return 'married'
    elif value in single_keywords:
        return 'single'
    elif value in divorced_keywords:
        return 'divorced'

In [248]:
user_info['marital_status'].unique()

array(['Married', 'cưới', 'Single', 'ly hôn', 'Divorced', nan,
       'Unmarried', 'độc thân'], dtype=object)

In [249]:
user_info['marital_status'] = user_info['marital_status'].apply(marital_status_mapping)
user_info['marital_status'].unique()

array(['married', 'single', 'divorced', nan], dtype=object)

In [250]:
user_info

,user_id,age,sex,phone,job,carrier,marital_status
0,376517,44.0,female,******6654,thuc tap sinh giao duc,other,married
1,234512,39.0,female,NaN,NaN,vietnamobile,married
2,344532,39.0,female,******6296,devops engineer,other,married
3,186135,37.0,female,******1502,thuc tap sinh giao duc,vietnamobile,married
4,30230,38.0,female,******4966,devops engineer,viettel,married
...,...,...,...,...,...,...,...
424165,395814,26.0,female,******8563,kien truc su phan mem,vinaphone,married
424166,245950,NaN,male,*****3449,tu van khach hang,other,divorced
424167,208016,NaN,NaN,******7187,truong phong kinh doanh,viettel,married
424168,272535,45.0,male,*******0404,duoc si,vinaphone,married


In [251]:
user_info.to_csv(r'cleaned_data/user_info.csv', index=False)